# search parameter space for reasonable to allow 1C discharge then 0.3C charge: D_cross, D_ec 

In [1]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import traceback

In [2]:
import imageio.v2 as imageio

In [3]:
V_max = 4.2;        V_min = 2.5;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [4]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a6_scan_Dec_Dcross/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [5]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);
Para_0['EC transference number'] =     -1.4
Para_0['Cation transference number'] =     0.28

Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = "[input]"
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   "[input]"
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =   "[input]" 

c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =(
    Para_0['Electrolyte conductivity [S.m-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff))
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =(
    Para_0['Electrolyte diffusivity [m2.s-1]'](c_e_ddiff,c_EC_ddiff, T_ddiff))

using EC wo refill for Li+


Xi =  list(np.linspace(-1.8, -1.0, 10))
print(type(Xi))

In [6]:
Sim_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)  

D_cross = [0, 1.5e-16, 1.5e-14, 1.5e-12, 1.5e-10, 1.5e-9];
D_ec = [5e-10, 5e-12]
Set_succ = []; Set_fail = []; ErrorMess = [];
index = 0
for D_ec_i in D_ec:
    for D_cross_i in D_cross:
        index = index+1
        try:
            Sol_ddiff    = Sim_ddiff.solve(
                inputs={
                    "Typical EC Lithium ion cross diffusivity [m2.s-1]":D_cross_i,
                    "EC Lithium ion cross diffusivity [m2.s-1]":D_cross_i,
                    "EC diffusivity in electrolyte [m2.s-1]":D_ec_i,});
            font = {'family' : 'Times New Roman','size': 18}
            mpl.rc('font', **font)
            output_variables3 = [
                "Terminal voltage [V]",   
                "EC concentration [mol.m-3]",
                "Electrolyte concentration [mol.m-3]",
                "c(EC) over c(Li+)",
                "Electrolyte conductivity [S.m-1]",
                "Electrolyte diffusivity [m2.s-1]",
            ]
            quick_plot = pybamm.QuickPlot([
                Sol_ddiff
                ], output_variables3,
                variable_limits='fixed',time_unit='hours',n_rows=2,
                figsize = (16,10)) #     spatial_unit='mm',
            quick_plot.dynamic_plot();
            quick_plot.create_gif(
                number_of_images=10, duration=2,
                output_filename=BasicPath + Target+str(index)+" Dcross="+str(D_cross_i)+" Dec="+str(D_ec_i) + ".gif")
        except:
            str_error = traceback.format_exc()  
            ErrorMess.append(str_error)
            print("Fail for: ", 
            "D_cross=",D_cross_i, "D_ec=", D_ec_i );
            Set_fail.append([D_cross_i,D_ec_i])

        else:
            print("Succeed for: ", 
            "D_cross=",D_cross_i, "D_ec=", D_ec_i );
            Set_succ.append([D_cross_i,D_ec_i])


psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
The residual routine or the linear setup or solve routine had a recoverable error, but IDACalcIC was unable to recover.
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian failed
psetup failed: .../casadi/interfaces/sundials/idas_interface.cpp:849: Calculating Jacobian faile

Fail for:  D_cross= 0 D_ec= 5e-10


interactive(children=(FloatSlider(value=0.0, description='t', max=4.135966025778702, step=0.04135966025778701)…

Succeed for:  D_cross= 1.5e-16 D_ec= 5e-10


interactive(children=(FloatSlider(value=0.0, description='t', max=4.135966247695979, step=0.04135966247695979)…

...

Succeed for:  D_cross= 1.5e-14 D_ec= 5e-12


interactive(children=(FloatSlider(value=0.0, description='t', max=4.138806719726431, step=0.04138806719726431)…

Succeed for:  D_cross= 1.5e-12 D_ec= 5e-12
Fail for:  D_cross= 1.5e-10 D_ec= 5e-12


At t = 1.46324e-006, , mxstep steps taken before reaching tout.
At t = 1.51866e-006 and h = 4.13385e-017, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.62486e-006 and h = 5.84566e-024, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.47964e-006 and h = 8.11659e-021, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 1.70839e-006 and h = 3.85936e-017, the corrector convergence failed repeatedly or with |h| = hmin.
2022-10-11 15:57:59.429 - [ERROR] callbacks.on_experiment_error(226): Simulation error: Maximum number of decreased steps occurred at t=0.0. Try solving the model up to this time only or reducing dt_max (currently, dt_max=3.5458164923888056) and/or reducing the size of the time steps or period of the experiment.


Fail for:  D_cross= 1.5e-09 D_ec= 5e-12


print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

output_variables3 = [
    "Terminal voltage [V]",   
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,
    variable_limits='fixed',time_unit='hours',n_rows=2,
    figsize = (16,10)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
quick_plot.create_gif(
    number_of_images=10, duration=2,
    output_filename=BasicPath + Target+str(index)+" Dcross="+str(D_cross_i)+" Dec="+str(D_ec_i) + ".gif")